In [1]:
import os
import re
import traceback

In [2]:
import pickle
import time
import numpy as np
import pymysql

In [5]:
from unitils import get_ans,ans_clean,pre_clean,f1

In [6]:
from unitils import match_type_cur,get_entry,from_value,get_type

In [7]:
SYNONYM_MAP = {
    '丈夫': '丈夫配偶',
    '老公': '丈夫配偶',
    '妻子': '妻子配偶',
    '夫人': '夫人妻子配偶',
    '媳妇': '妻子配偶',
    '老家': '老家籍贯',
    '故乡': '故乡籍贯',
    '家乡': '家乡籍贯', 
    '英文':'外文英文',
    '西班牙语':'外文西班牙语',
    '贡献' : '贡献主要成就',
    '功绩' : '功绩主要成就',
    '丰功伟绩' : '丰功伟绩主要成就',
    '来源' : '来源出处',
    '在金庸小说《天龙八部》中':''
}

In [8]:
SAMPLE_LENGTH = 20

# 融合答案

## 可用字符确定的类型

In [9]:
type_hint = {}
type_dict = {}

In [10]:
type_dict["国家"] = "哪些地方 哪国 在哪里 哪个国家 什么地方 帝国 时代 朝代"
type_hint["国家"] = "<地点><国家><二战参战国家><西半球><北美><北半球><历史机构><外国历史>"

In [11]:
type_dict["地址"] = "哪个地方 景区 分布区域 哪里 城市"
type_hint["地址"] = "<旅游><行政区划><自然地理><旅游><自然><景观景点><国家><历史><地点><外国历史>"

In [12]:
#type_dict["朝代"] = "什么时候 什么朝代 哪个时代"

In [13]:
type_dict["人物"] = "足球 花滑运动员 人物 是谁 经济学家 女歌手 科学家 哲学家 作家 物理学家 模特 谁 演员 企业家 天文学家 政治家 外交家 歌手 哪位 女性"
type_hint["人物"] = "<经济人物><人物><话题人物><文学形象>\
    <演员><娱乐人物><歌手><导演><行业人物><经济人物><互联网人物><80年代演员><香港演员><人物><政治人物><元首>\
    <虚拟人物><动漫形象><行业人物><人物><文学>"

In [14]:
type_dict["作品"] = "作品 电视剧 电影 书籍 专辑 著作 代表作 出自哪里 哪几部电视剧"
type_hint["作品"] = "<书籍><出版物><电视剧><中国文学><诗词><文学><音乐作品><音乐><单曲>"

In [15]:
type_dict["大学"] = "毕业 哪所院校 高校"
type_hint["大学"] = "<大学><中国大学><学校><外国大学>"

In [16]:
def find_ans_type(seq):
    tag = {}   
    for key in type_dict:
        for line in type_dict[key].split(" "):
            if line in seq:
                tag[line] = (type_hint[key],seq.rindex(line))
                break
    out = list(tag.items())
    out.sort(key = lambda x:x[1][1])
    if out:
        return out[-1][1][0]
    else:
        return ""

In [17]:
def att_type(ans_seq,tag):
    words = ans_seq.split("\t")
    #print(words)
    for word in words:
        word = word.replace("'","\\'")
        if not word:
            continue
        try:
            sss = get_type(word,match_type_cur)
        except pymysql.ProgrammingError:
            print(word)
            continue
        for row in sss:
            if row[3].strip(" .") in tag:
                return 1
    return 0

## 需要正则确定的类型

### 网址

In [18]:
site_pattern = re.compile("(\.com|\.net|\.cn|\.org|\.gov|\.edu)")

In [19]:
site_hint = "网址 官网 官方网站"

In [20]:
def is_site(seq):
    if site_pattern.findall(seq):
        return 1
    return 0

### 长度

In [21]:
len_hint = "有多长 海拔 有多高"

In [22]:
len_pattern = re.compile("[0-9一二三四五六七八九十百十千万亿\.米kmKMcm]+")

### 数量

In [23]:
num_hint = "总数 有几个 是多少 有多少 价格是多少"

In [24]:
num_pattern = re.compile("[0-9一二三四五六七八九十百十千万亿]+")

### 时间

In [25]:
time_hint = "什么时候 哪一年 哪一天 何时" 
#1890-07-29 1953-06

In [26]:
#   ->哪个城市 
time_pattern = re.compile("(\d{2,4}年\d{1,2}月\d{1,2}日|\d{2,4}年\d{1,2}月|\d{1,2}月\d{1,2}日\
|\d{2,4}-\d{1,2}-\d{0,2}|\d{2,4}-\d{1,4}|\d{4}|-\d{3})")

## 读取待融合文件

In [27]:
result = open("result_ss.txt","rb").read().decode("utf8").split("\r\n")
# 12 16

In [28]:
question = open("task4coqa_test.questions","rb").read().decode("utf8").split("\r\n")

In [29]:
#result_2 = open("result_9.txt","rb").read().decode("utf8").split("\n")
result_2 = open("result.test.txt","rb").read().decode("utf8").split("\n")

In [30]:
result_3 = open("result_new.txt","rb").read().decode("utf8").split("\n")

In [31]:
path_txt = open("path.test.txt","rb").read().decode("utf8").split("\n")

In [32]:
re_submit = result_2

## 读取感知机中间结果

In [33]:
with open("total_test_feature_07_20_13_11.bin","rb") as f: 
    total_test_feature = pickle.load(f)

In [34]:
with open("nnn_07_20_10_06.bin","rb") as f: 
    nnn = pickle.load(f)

## 开始融合

In [33]:
i = 0
for que,a1,a2,a4 in zip(question,result,result_2,result_3):
    
    if len(set(a1.split("\t")).intersection(set(a2.split("\t")))) == 0 :
        tag = find_ans_type(que)
        if not tag:
            tag = "<PAD>"
        q_index = i
        a1_tt = att_type(a1,tag)
        a2_tt = att_type(a2,tag)
        a4_tt = att_type(a4,tag)
        
        index = nnn.argmax(axis = 1)[i]
        other = total_test_feature[i][1][abs(index-1)+1]
        a3 = "\t".join(other[0])
        other_tt = att_type(a3,tag)
        if q_index == 92:
            print(a1,a2,a3)
        # 有且仅有一个命中了类型  
        if abs(a1_tt - a2_tt)!= 0:
            if a1_tt == 1:
                re_submit[i] = a1.split("\t")
            else:
                re_submit[i] = a2.split("\t")
                
        # 如果两个都命中类型 或 都没有命中类型      
        else:
            # 投票决定
            if a1_tt == 0 and other_tt == 1:
                re_submit[i] = a3.split("\t")
            elif set(a3) == set(a1) or set(a4) == set(a1) and a1!= [""]:
                re_submit[i] = a1.split("\t")
            elif set(a3) == set(a2) or set(a2) == set(a4) and a2!= [""]:
                re_submit[i] = a2.split("\t")
            elif set(a3) == set(a4) and a3!= [""]:
                re_submit[i] = a3.split("\t")
            elif a1_tt == 0 and a4_tt == 1:
                re_submit[i] = a4.split("\t")
                
            else: # 理论上我们要比较path了,这里将使用新版本的path
                if a2:
                    re_submit[i] = a2.split("\t")
                else:
                    re_submit[i] = a1.split("\t")
                one_path = set(question[q_index]) - set(str(total_test_feature[q_index][1][1])) - set("1234567890？q:：什么的是全")
                two_path = set(question[q_index]) - set(str(total_test_feature[q_index][1][2]))  - set("1234567890？q:：什么的是全")
                new_path = set(question[q_index]) - set(path_txt[q_index]) - set("1234567890？q:：什么的是全")
                if i== 117:
                    print(len(a1),len(a2),len(a3))
                    print(index)
                    print(a1,a2,a3)
                    print(one_path)
                if index == 0:
                    bb = np.array(list(map(len,[one_path,new_path])))
                else:
                    # 这里本来是one_path
                    bb = np.array(list(map(len,[two_path,new_path])))
                if bb.max()-bb.min()>=4 or bb.max()==0:
                    if bb[0] <= bb[1]:
                        re_submit[i] = a1.split("\t")
                    else:
                        re_submit[i] = a4.split("\t")
                    #print("-"*130)
                    #print(i,que)
                
        
        # 强制类型修正
        
        # 长度型答案
        for row in len_hint.split(" "):
            if row in que:
                bb = np.array(list(map(lambda x: len(len_pattern.findall(x)),[a1,a2,a3])))
                re_submit[i] = [[a1,a2,a3][int(bb.argmax())]]
                #print("len",re_submit[i])
        # 数字答案
        for row in num_hint.split(" "):
            if row in que:
                bb = np.array(list(map(lambda x: len(len_pattern.findall(x)),[a1,a2,a3,a4])))
                if sum(bb >0) == 1:
                    re_submit[i] = [[a1,a2,a3,a4][int(bb.argmax())]]
        # 网站型答案
        for row in site_hint.split(" "):
            if row in que:
                #print("+"*130)
                #print("len",que)
                bb = np.array(list(map(lambda x: len(site_pattern.findall(x)),[a1,a2,a3])))
                re_submit[i] = [[a1,a2,a3][int(bb.argmax())]]
        # 地址型问题        
        if "地址" in que:
            re_submit[i] = a4.split("\t")
        # 时间型答案
        if re_submit[i] == [''] or "".join(re_submit[i]) in que:
            #print(i,que)
            re_submit[i] = a1.split("\t")
            #print("change",a1)
    else:
        re_submit[i] = a2.split("\t")
    i+=1

In [34]:
with open("result.txt","wb") as f:
    for line in re_submit:
        #print(line)
        line = "\t".join(line) + "\r\n"
        f.write(line.encode("utf8"))